# Guessing transition states with force fireld interpolation

Finding transition states is of crucial importance for computational chemistry, and there are many methods for doing so. However, most of these methods require some initial guess of the transition state structure. If the guess is not good enough, the method might not converge. Reaction Force Field interpolation allows for generating such guesses at a very cheap computational level by relying mostly on molecular mechanics.

Force field interpolation works by describing the reactant and product states with classical force fields $E_{1}$ and $E_{2}$ respectively. By taking a linear combination 

$$V(\lambda) = (1-\lambda)E_{1} + \lambda E_{2}$$

one can crudely describe the Potential Energy Surface (PES) along the reaction. By doing an MM optimisation of the structure at various values of $\lambda$, the relative MM energy can be obtained. The optimised structure can also be used to perform a single-point quantum mechanical (QM) calculation, which will give an more accurate energy. The structure at which these energies are highest is a good guess for the transition state structure. This workflow can be executed with just a couple lines of code. Let's consider the Bromine substitution of Ethyl chloride:

$$\mathrm{C_2H_5Cl + Br^- \rightarrow C_2H_5Br + Cl^-}$$

The following code will perform the entire workflow and visualise the results:

In [1]:
import veloxchem as vlx

In [ ]:
import veloxchem as vlx

rea1 = vlx.Molecule.read_smiles("[Br]")
rea2 = vlx.Molecule.read_smiles("CCCl")
rea1.set_charge(-1)

pro1 = vlx.Molecule.read_smiles("[Cl]")
pro2 = vlx.Molecule.read_smiles("CCBr")
pro1.set_charge(-1)

tsguesser = vlx.TransitionStateGuesser()
results = tsguesser.find_TS([rea1, rea2], [pro1, pro2])
tsguesser.show_results(results)

* Info * Building forcefields. Disable mute_ff_build to see detailed output.                                              
                                                                                                                          
                                                     Reaction summary                                                     
                                                    1 breaking bonds:                                                     
                                       ReaType  ProType  ID - ReaType  ProType  ID                                        
                                          c3       c3    3  -    cl       cl    4                                         
                                                                                                                          
                                                     1 forming bonds:                                                     
                

interactive(children=(SelectionSlider(description='Lambda', index=10, options=(0.0, 0.05, 0.1, 0.15, 0.2, 0.25…

To better understand the workflow and it's limitations, let's go through it step by step.

## Step by step calculation of $S_N 2$ reaction



To create interpolated forcefields, we need to provide reactant and product structures as molecule objects. The program will then calculate partial charges, but it is also possible to provide those manually. The program figures out which bonds are broken and formed. It then does an MM optimisation of the combined reactand and product systems which will be used as starting points for the scan.

In [4]:
rea1 = vlx.Molecule.read_smiles("[Br]")
rea2 = vlx.Molecule.read_smiles("CCCl")
rea1.set_charge(-1)

pro1 = vlx.Molecule.read_smiles("[Cl]")
pro2 = vlx.Molecule.read_smiles("CCBr")
pro1.set_charge(-1)
tsguesser = vlx.TransitionStateGuesser()

tsguesser.build_forcefields([rea1, rea2], [pro1, pro2])

* Info * Building forcefields. Disable mute_ff_build to see detailed output.                                              
                                                                                                                          
                                                     Reaction summary                                                     
                                                    1 breaking bonds:                                                     
                                       ReaType  ProType  ID - ReaType  ProType  ID                                        
                                          c3       c3    3  -    cl       cl    4                                         
                                                                                                                          
                                                     1 forming bonds:                                                     
                

The optimised structures can be inspected as follows:

In [5]:
tsguesser.reactant.molecule.show()
tsguesser.product.molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

If this looks reasonable, we can proceed to scan the reaction. By default, the reaction is split up in 21 steps. On every step, the structure is minimised to the new forcefield, then sampled for 1000 steps at 600K, and then optimised again. If the value of $E1$ decreases (or $E2$ increases), this indicates that the obtained structures are could be part of different valleys in the energy landscape. The program will then try to do a conformer search to find the lowest lying conformer.

In [6]:
mm_results = tsguesser.scan_mm()

                                                                                                                          
Starting MM scan for lambda values [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
                                                                                                                          
                                                      MM parameters:                                                      
                                            MD steps:                    1000                                             
                                            MD temperature:             600 K                                             
                                            MD step size:            0.001 ps                                             
                                            folder name:   ts_data_1758618280                                  

The results can then be visualised.

In [7]:
tsguesser.show_results(mm_results)

interactive(children=(SelectionSlider(description='Lambda', index=10, options=(0.0, 0.05, 0.1, 0.15, 0.2, 0.25…

While the MM energies give a reasonable description of the energetic contributions from coloumbic and van der Waals interactions, they do not capture bond breaking and formation. To get a better estimate of the energy, we can perform single-point QM calculations on the MM optimised structures. Here, we use DFT with the B3LYP functional and the 'def2-SVP' basis set which are the default settings.

In [8]:
tsguesser.scf_xcfun = 'b3lyp'
tsguesser.scf_basis = 'def2-svp'
scf_results = tsguesser.scan_scf()

* Info * Disable mute_scf to see detailed output.                                                                         
                                                                                                                          
                                                    Starting SCF scan                                                     
                                                                                                                          
                                                     SCF parameters:                                                      
                                                 basis:         def2-svp                                                  
                                                 DFT xc fun:       b3lyp                                                  
                                                                                                                          
                

In [9]:
tsguesser.show_results(scf_results)

interactive(children=(SelectionSlider(description='Lambda', index=11, options=(0.0, 0.05, 0.1, 0.15, 0.2, 0.25…

And these results can be visualised as well. Note how in this (simple) example, the peak of the MM energies coincides with the SCF peak. In more complex reactions, this is not always the case, but it is usually close and the MM peak might already be a good enough guess.

Then we can proceed with a proper QM transition state optimisation.

In [ ]:
basis = vlx.MolecularBasis.read(tsguesser.molecule, "def2-svp")
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_drv.xcfun = "b3lyp"
scf_results = scf_drv.compute(tsguesser.molecule, basis)
scf_opt_drv = vlx.OptimizationDriver(scf_drv)
scf_opt_drv.ostream.mute()
scf_opt_drv.transition = True  #We set the option to find transition states
opt_results = scf_opt_drv.compute(tsguesser.molecule, basis, scf_results)
vlx.Molecule.read_xyz_string(opt_results['final_geometry']).show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Conformer searching

Some systems might have many conformers along the PES, and in order to find the correct transition state, it is important to find the lowest lying conformer. When `peak_conformer_search` is set to `True`, a conformer search will be performed at the peak of the PES over a range of $\lambda$ values. It is also possible to automatically perform a conformer search when a discontinuity in the PES is detected by setting `discont_conformer_search` to `True`. Lastly, setting `force_conformer_search` to `True` will perform a conformer search at every $\lambda$ value. All of these functions are turned off by default because they are more time consuming. Depending on the system and the use case, the user has to decide which options are appropriate. In the following example, the longer carbon backbone gives the system more conformational freedom. Rescanning the entire PES is not necessary, but performing a conformer search at the peak is useful to obtain the correct conformer for the transition state guess.

In [2]:
rea1 = vlx.Molecule.read_smiles("[Br]")
rea2 = vlx.Molecule.read_smiles("CC(C)CC(Cl)C")
rea1.set_charge(-1)

pro1 = vlx.Molecule.read_smiles("[Cl]")
pro2 = vlx.Molecule.read_smiles("CC(C)CC(Br)C")
pro1.set_charge(-1)

tsguesser = vlx.TransitionStateGuesser()

tsguesser.build_forcefields([rea1, rea2], [pro1, pro2])

tsguesser.peak_conformer_search = True
tsguesser.peak_conformer_search_range = 2
results = tsguesser.scan_mm()
tsguesser.show_results(results)

* Info * Building forcefields. Disable mute_ff_build to see detailed output.                                              
                                                                                                                          
                                                     Reaction summary                                                     
                                                    1 breaking bonds:                                                     
                                       ReaType  ProType  ID - ReaType  ProType  ID                                        
                                          c3       c3    6  -    cl       cl    7                                         
                                                                                                                          
                                                     1 forming bonds:                                                     
                

interactive(children=(SelectionSlider(description='Lambda', index=10, options=(0.0, 0.05, 0.1, 0.15, 0.2, 0.25…

## Forced bond breaking

To correctly interpolate between the reactant and the product, each atom in the reactant and product need to match. This mapping is detected automatically, but can be provided manually. From this mapping the breaking and forming bonds are derived. It is even possible to specify bonds that should be forced to break which are then not allowed to reconnect. This is useful for example in certain catalysis reactions, where both the reactant and the product contain the same catalyst molecule. The optimal mapping would not break or form any bonds with these molecules, but the reaction of interest should involve this catalyst. The following example demonstrates this with methanol catalysing the proton transfer of a larger complex. In order to run the example faster, the partial charges are pre-computed and provided manually.

In [ ]:
evb = vlx.EvbDriver()

q_rea = [[
    -0.63376,
    0.155898,
    0.40852,
    0.023114,
    0.023114,
    0.023114,
],
         [
             -0.053121, -0.224161, -0.146708, -0.198581, -0.104851, 0.057048,
             0.211108, -0.00552, 0.582421, -0.578475, -0.239007, -0.200459,
             0.076475, 0.046494, -0.014331, -0.064532, -0.241586, -0.879622,
             0.102603, 0.135941, 0.12827, 0.137736, 0.085068, 0.010794,
             0.056581, 0.118934, 0.118934, 0.118934, 0.071642, 0.071642,
             0.084769, 0.084769, 0.084769, 0.068894, 0.068894, 0.068894,
             0.11978, 0.11978, 0.11978
         ]]
q_pro = [
    [
        -0.637505,
        0.152377,
        0.411487,
        0.024547,
        0.024547,
        0.024547,
    ],
    [
        -0.088958, -0.19184, -0.147759, -0.194697, -0.060166, 0.060322,
        0.057387, -0.279844, 0.484962, -0.274851, -0.152366,
        -0.7389260000000002, 0.034466, -0.007965, -0.126105, -0.288341,
        -0.009113, -0.684542, 0.112558, 0.144675, 0.133772, 0.134584, 0.1315,
        0.137143, 0.100035, 0.100035, 0.100035, 0.071572, 0.071572, 0.100344,
        0.100344, 0.100344, 0.139586, 0.139586, 0.139586, 0.066736, 0.066736,
        0.066736, 0.450857
    ],
]

rea1 = vlx.Molecule.read_smiles("OC")
rea2 = vlx.Molecule.read_smiles("C1=CC=CC=C1C(C(C(=O)OC)C[N+](C)(C)C)([O-])")

pro1 = vlx.Molecule.read_smiles("OC")
pro2 = vlx.Molecule.read_smiles("C1=CC=CC=C1C(C(=C(OC)[O-])C[N+](C)(C)C)O")

rea_charges = q_rea
pro_charges = q_pro
breaking_bonds = {(0, 2)}
tsguesser = vlx.TransitionStateGuesser()
tsguesser.build_forcefields(
    [rea1, rea2],
    [pro1, pro2],
    breaking_bonds={(1, 3)},
    reactant_partial_charges=rea_charges,
    product_partial_charges=pro_charges,
)

* Info * Building forcefields. Disable mute_ff_builder to see detailed output.                                            
                                                                                                                          
                                                     Reaction summary                                                     
                                                    2 breaking bonds:                                                     
                                       ReaType  ProType  ID - ReaType  ProType  ID                                        
                                          c3       c2    14 -    hc       ho    31                                        
                                          oh       oh    1  -    ho       ho    3                                         
                                                                                                                          
                

The output shows that the program found the correct forming and breaking bonds. We can then proceed with the mm scan.

In [ ]:
mm_results = tsguesser.scan_mm()
tsguesser.show_results(mm_results)

                                                                                                                          
                                                 Starting initial MM scan                                                 
                                                                                                                          
                                                      MM parameters:                                                      
                                                mm steps:             1000                                                
                                                mm temperature:      600 K                                                
                                                mm step size:     0.001 ps                                                
                                                folder name:       ts_data                                                
                

interactive(children=(SelectionSlider(description='Lambda', index=10, options=(0.0, 0.05, 0.1, 0.15, 0.2, 0.25…

## Available parameters and options

There are a bunch more options available to customise the TS-guesser. Below is a complete list of all options with their default values.



In [ ]:
water = vlx.Molecule.read_smiles('O')

rea = vlx.Molecule.read_smiles('C(C(=O)O)([O-])')
pro = vlx.Molecule.read_smiles('C(=C(O)[O-])O')

tsguesser = vlx.TransitionStateGuesser()
tsguesser.mute_ff_build = False

tsguesser.ffbuilder.optimize_mol = False  # If True, does an xtb optimization of every provided molecule object before reparameterisation.
tsguesser.ffbuilder.reparameterize_bonds = False  # If True, reparameterizes unknown bond force constants with the Seminario method.
tsguesser.ffbuilder.reparameterize_angles = False  # If True, reparameterizes unknown angle force constants with the Seminario method.
tsguesser.ffbuilder.optimize_ff = True  # If True, does an mm optimization of the combined reactant and product after reparameterisation.
tsguesser.ffbuilder.mm_opt_constrain_bonds = True  # If true, constrains nonbonded interactions of atoms involved in breaking/forming bonds during the mm optimization.
tsguesser.ffbuilder.water_model = 'spce'  # Water model to use in case a water molecule is provided
tsguesser.ffbuilder.product_mapping = None  # Optional mapping of product atoms to reactant atoms
tsguesser.ffbuilder.mute_scf = False  # If True, mutes the scf output during xtb, resp and hessian calculations
tsguesser.ffbuilder.skip_reaction_matching = False  # If True, skips the matching of product atoms to the reactant and assumes the order of the given structures to be correct
tsguesser.ffbuilder.hessian_basis = 'def2-SV_P_'  # Basis set for hessian calculation. For now cannot contain f-functions
tsguesser.ffbuilder.hessian_xc_fun = 'B3LYP'  # xc functional for hessian calculation

tsguesser.lambda_vec = [
    0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65,
    0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1
]  # Lambda values to scan
tsguesser.scf_xcfun = "b3lyp"  # xc functional for scf calculations
tsguesser.scf_basis = 'def2-svp'  # Basis set for scf calculations
tsguesser.mute_scf = True  # If True, mutes the scf output during scf calculations
tsguesser.mute_ff_build = True  # If True, mutes the output from the force field builder
tsguesser.mm_temperature = 600  # Temperature for mm md runs in K
tsguesser.mm_steps = 1000  # Number of steps for mm md runs
tsguesser.mm_step_size = 0.001  # Step size for mm md runs in ps
tsguesser.save_mm_traj = False  # If True, saves the mm md trajectory as an xyz file in the ts_data folder
tsguesser.scf_drv = None  # Custom scf driver to be used for all scf calculations
tsguesser.folder_name = 'ts_data'  # Folder name for all ts data
tsguesser.force_conformer_search = False  # If True, forces a conformer search on all lambda values
tsguesser.discont_conformer_search = True  # If False, performs a conformer search where E2 is increasing or E1 is decreasing, which indicates a discontinuity in the PES
tsguesser.peak_conformer_search = False  # If True, performs a conformer search at the peak of the PES
tsguesser.peak_conformer_search_range = 1  # Range of lambda values around the peak to perform the conformer search
tsguesser.conformer_steps = 10000  # Number of steps for the conformer search md runs
tsguesser.conformer_snapshots = 10  # Number of snapshots to take during the conformer search
tsguesser.results_file = 'ts_results.h5'  # File to save the results

tsguesser.find_TS(
    [water, rea],  # reactants
    [water, pro],  # products
    # If an scf calculation should be done. Defaults to True
    scf=True,
    # Set of tuples with indices (1-based) of bonds that have to break and are not allowed to re-connect
    breaking_bonds={(1, 3)},
    # Optional partial charges for the reactant. Should be a list of list of floats matching the list of reactants
    reactant_partial_charges=None,
    # Optional partial charges for the product. Should be a list of list of floats matching the list of products
    product_partial_charges=None,
    # Optional hessians for the reactant. Should be a list of np.arrays matching the list of reactants
    reactant_hessians=None,
    # Optional hessians for the product. Should be a list of np.arrays matching the list of products
    product_hessians=None,
    # Total reactant multiplicity to override override the guess
    reactant_total_multiplicity=-1,
    # Total product multiplicity to override override the guess
    product_total_multiplicity=-1,
)